Importando as bibliotecas utilizadas

In [ ]:
import pandas as pd
import requests
import json

API e criação do dataframe

In [ ]:
url_2 = 'https://dadosabertos.aneel.gov.br/api/3/action/datastore_search?resource_id=b1bd71e7-d0ad-4214-9053-cbd58e9564a7&limit=32000&filters={"DscPorte":"Minigeracao"}'
response_2 = requests.get(url_2)

In [ ]:
resp_json = response_2.json()
result = resp_json.get('result')
records = result.get('records')
Empreendimentos = pd.DataFrame(records)

Eliminação de colunas indesejadas do dataframe

In [ ]:
to_drop2 = ['_id','AnmPeriodoReferencia','DatGeracaoConjuntoDados','NumCNPJDistribuidora','NomAgente','CodClasseConsumo',
            'CodSubGrupoTarifario','codUFibge','codRegiao','CodMunicipioIbge','NomMunicipio','CodCEP','SigTipoConsumidor',
            'NumCPFCNPJ','NomTitularUC','CodGD','SigModalidadeEmpreendimento','DscFonteGeracao','NomSubEstacao GEO',
            'NumCoordNSub GEO', 'NumCoordEUC', 'NumCoordNUC','NomSubEstacao','NumCoordESub', 'NumCoordESub GEO',
            'NumCoordNSub','NomRegiao','DscPorte','NumLatitude', 'NumLongitude']
Empreendimentos.drop(to_drop2, axis = 1, inplace = True)

Manipulação de tipos e conversão de kW para MW

In [ ]:
Empreendimentos.MdaPotenciaInstaladaKW = Empreendimentos.MdaPotenciaInstaladaKW.replace(',','.', regex = True)
Empreendimentos.MdaPotenciaInstaladaKW = pd.to_numeric(Empreendimentos.MdaPotenciaInstaladaKW)/1000
Empreendimentos.MdaPotenciaInstaladaKW = Empreendimentos.MdaPotenciaInstaladaKW.round(4)
Empreendimentos.MdaPotenciaInstaladaKW = Empreendimentos.MdaPotenciaInstaladaKW.astype(str).str.replace('.',',')

In [ ]:
Empreendimentos.QtdUCRecebeCredito = pd.to_numeric(Empreendimentos.QtdUCRecebeCredito)
Empreendimentos.DthAtualizaCadastralEmpreend = pd.to_datetime(Empreendimentos.DthAtualizaCadastralEmpreend)

Criação de colunas no dataframe, a partir do mapeamento de outras colunas

In [ ]:
fonte = {'UFV':'Solar',
         'UTE':'Térmica',
         'CGH':'Hídrica',
         'EOL':'Eólica'}
Empreendimentos['Fonte'] = Empreendimentos['SigTipoGeracao'].map(fonte)

In [ ]:
modalidade = {'Com Microgeracao ou Minigeracao distribuida':'Autoconsumo local',
              'Caracterizada como Autoconsumo remoto':'Autoconsumo remoto',
              'Caracterizada como Geracao compartilhada':'Geração compartilhada',
              'Integrante de empreendimento de Multiplas UC':'Condomínios'
              }
Empreendimentos.DscModalidadeHabilitado = Empreendimentos.DscModalidadeHabilitado.map(modalidade)

Preenchimento de linhas vazias de acordo com a moda estatística

In [ ]:
Empreendimentos.fillna({'DscModalidadeHabilitado':'Autoconsumo local'}, inplace= True)

Adequação de valores

In [ ]:
Empreendimentos.loc[Empreendimentos['DscClasseConsumo'].isin(['Serviço Público','Iluminação pública']),'DscClasseConsumo'] = 'Outros'

Renomeando colunas do dataframe

In [ ]:
rename_2 = {'SigAgente':'Concessionária',
           'DscClasseConsumo':'Classe Consumidor',
           'DscSubGrupoTarifario':'Grupo Tarifário',
           'SigUF':'Estado',
           'DthAtualizaCadastralEmpreend':'Data Cadastrada',
           'QtdUCRecebeCredito':'Qtde UC associadas',
           'SigTipoGeracao':'Tipo Geração',
           'DscModalidadeHabilitado':'Modalidade',
           'MdaPotenciaInstaladaKW':'Potência Instalada (MW)'}
Empreendimentos.rename(rename_2, axis = 1, inplace = True)